In [ ]:
%pylab inline
import xarray as xr
from matplotlib import animation, rc
from parcels.plotting import create_parcelsfig_axis, cartopy_colorbar
from parcels import plotTrajectoriesFile
from matplotlib import colors
from progressbar import ProgressBar
from os import environ

In [ ]:
environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
filename = 'pumice_wind1pct_stokes_hycom_delayedtime.nc'
pfile = xr.open_dataset(str(filename), decode_cf=True)
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)

pfile.close()
pbar = ProgressBar()
for p in pbar(range(lon.shape[1])):
    lon[:, p] = [ln if ln < 180 else ln - 360 for ln in lon[:, p]]

In [ ]:
pylab.rcParams['figure.figsize'] = (15, 10)

plt, fig, ax, cartopy = create_parcelsfig_axis(spherical=True)
_, _, _, cs = plt.hist2d(lon[~np.isnan(lon)], lat[~np.isnan(lat)], bins=[180, 75], norm=colors.LogNorm())
plt.scatter(lon[:, 0], lat[:, 0], c='r', s=10, transform=cartopy.crs.Geodetic())
cartopy_colorbar(cs, plt, fig, ax)
ax.set_title('Particle histogram for %s' % filename)
plt.show()

In [ ]:
pylab.rcParams['figure.figsize'] = (8, 4)
plottimes = time[0,0] + np.arange(0, np.timedelta64(365*3, 'D'), np.timedelta64(5, 'D'))

b = time == plottimes[0]
plt, fig, ax, cartopy = create_parcelsfig_axis(spherical=True)
scat = ax.scatter(lon[b], lat[b], s=20, color='k', transform=cartopy.crs.Geodetic())

ttl = ax.set_title('Particles at time ' + str(plottimes[0]))
frames = np.arange(1, len(plottimes))

def animate(t):
    b = time == plottimes[t]
    scat.set_offsets(np.vstack((lon[b], lat[b])).transpose())
    ttl.set_text('Particle at time ' + str(plottimes[t]))
    return scat,

rc('animation', html='html5')
anim = animation.FuncAnimation(fig, animate, frames=frames, interval=100, blit=False)

from IPython.display import HTML
HTML(anim.to_html5_video())